# Import

In [1]:
import os
import numpy as np
import tensorflow as tf
import random
from gensim.models.word2vec import Word2Vec
import gc
import time
import pickle

# Load graph

In [2]:
with tf.gfile.FastGFile(os.path.join(
    '../../../data/inception-2015-12-05/classify_image_graph_def.pb'), 'rb') as f:
    graph_def = tf.GraphDef()
    graph_def.ParseFromString(f.read())
    _ = tf.import_graph_def(graph_def, name='')
g = tf.get_default_graph()

## Create own branch

In [3]:
with tf.name_scope('own'):
    y = tf.placeholder("float")
    x = g.get_tensor_by_name('pool_3/_reshape:0')
    w = tf.Variable(tf.random_normal(
        [int(x.get_shape()[-1]),300], stddev=float('1e-5')), name='weights')
    b = tf.Variable(tf.random_normal([1,300]), name='bias')
    y_pred = tf.add(tf.matmul(x, w), b, name='y_pred')
    cost = tf.reduce_sum(tf.square(y-y_pred),name='cost')
    cost_summary = tf.summary.scalar('cost', cost)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate=float('1e-5')).minimize(cost)
    
timestamp = time.strftime('%Y%m%d-%H%M', time.localtime())
tf.summary.FileWriter('./sums/'+timestamp, graph=g)

# Read data

In [4]:
# Images
image_path = '../../../data/train2014/'
image_list = os.listdir(image_path)

# Caption dictionaries
vec_dict = np.load('../../../data/word2vec_train.npy').item()
#with open('../../../data/caption_train.pkl', 'rb') as f:
#    string_dict = pickle.load(f)

# Train model

In [6]:
with tf.Session(graph=g) as sess:
    # Initialize writers
    train_writer = tf.summary.FileWriter('./sums/'+timestamp+'/train/', graph=g, flush_secs=20)
    test_writer = tf.summary.FileWriter('./sums/'+timestamp+'/test/', graph=g, flush_secs=20)
    
    sess.run([w.initializer, b.initializer])
    
    # Train
    print('TRAINING')
    for i, im in enumerate(image_list[:2000]):
        image = tf.gfile.FastGFile(image_path + im, 'rb').read()
        # Choose one of the five captions randomly
        r = random.randrange(len(vec_dict[im]))
        y_temp = vec_dict[im][r].reshape((1,300))

        sess.run(optimizer,feed_dict={'DecodeJpeg/contents:0':image, y:y_temp})
        train_writer.add_summary(sess.run(cost_summary,{'DecodeJpeg/contents:0':image, y:y_temp}), i)
        
        if i % 10 == 0:
            test_image = tf.gfile.FastGFile(image_path + image_list[-1], 'rb').read()
            y_test = vec_dict[image_list[-1]][1].reshape((1,300))
            test_writer.add_summary(sess.run(cost_summary,{'DecodeJpeg/contents:0':test_image, y:y_test}), i)
            print(i, end=' ')
    print('\n\n')
    
    # Save
    print('SAVING NEW DICTIONARY')
    new_vec_dict = {}
    
#    # Multi-threaded
#    def get_vec(im):
#        image = tf.gfile.FastGFile('../../../data/train2014/' + im, 'rb').read()
#        v = tf.get_default_session().run(y_pred,{'DecodeJpeg/contents:0':image})
#        global count
#        with count.get_lock():
#            count.value += 1
#            print(count.value)
#        return {im: v}
#    from multiprocessing import Pool, Value
#    with Pool(2) as pool:
#        count = Value("i", 0)
#        vec_map = pool.map(get_vec, image_list[-1000:])
#        for j in vec_map:
#            new_vec_dict.update(j)
            
    # Single-threaded
    for i, im in enumerate(image_list[2000:]):
        image = tf.gfile.FastGFile(image_path + im, 'rb').read()
        new_vec_dict[im] = sess.run(y_pred,{'DecodeJpeg/contents:0':image})
        if i % 10 == 0:
            print(i, end=' ')
    
    np.save('image_space.npy',new_vec_dict)
        

TRAINING
b'\n\x11\n\nown/cost_1\x15\x13R\xacC'
0 b'\n\x11\n\nown/cost_1\x15.\xcd\xa1C'
10 b'\n\x11\n\nown/cost_1\x15\xc5o\x97C'
20 b'\n\x11\n\nown/cost_1\x15\x82\xbf\x8dC'
30 b'\n\x11\n\nown/cost_1\x15v\xe5\x85C'
40 b'\n\x11\n\nown/cost_1\x15\xad\x92}C'
50 b'\n\x11\n\nown/cost_1\x158\x08nC'
60 b'\n\x11\n\nown/cost_1\x15p!`C'
70 b'\n\x11\n\nown/cost_1\x15\xe8\x9fSC'
80 

KeyboardInterrupt: 